In [1]:
import os

os.chdir('../..')

In [2]:
import utils

In [7]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

In [20]:
path = r"output\exp1.1\models\Wavelet+NN\best_loss_model.h5"

model = load_model(path)

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 864)]             0         
                                                                 
 dense (Dense)               (None, 128)               110720    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 23)                2967      
                                                                 
Total params: 113,687
Trainable params: 113,687
Non-trainable params: 0
_________________________________________________________________


In [19]:
from sklearn.metrics import roc_auc_score
from tensorflow.keras.metrics import AUC

y_pred_path = r"output\exp1.1\models\Wavelet+NN\y_test_pred.npy"

y_pred = np.load(y_pred_path, allow_pickle = True)

In [11]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 100
task = 'subdiagnostic'
experiment_name = 'exp1.1'

data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

This experiment has 23 classes


In [22]:
roc_auc_score(y_test, y_pred)

0.7014160465218776

In [17]:
from models.baseline_wavelet import *
import pickle

XF_test = get_ecg_features(X_test)
ss = pickle.load(open(r"output\exp1.1\models\Wavelet+NN\ss.pkl", "rb"))
XFT_test = ss.transform(XF_test)


100%|██████████| 2158/2158 [03:02<00:00, 11.85it/s]


In [23]:
preds = model.predict(XFT_test)

68/68 [==============================] - 0s 2ms/step


In [25]:
roc_auc_score(y_test, preds)

0.798775609060111